## Label and Transform content locally

This notebook walks through training a transformation model and redacting PII locally in your environment.

Follow the instructions here to set up your local environment: https://docs.gretel.ai/environment-setup

Prerequisites:

- Python 3.9+ (`python --version`).
- Ensure that Docker is running (`docker info`).
- The Gretel client SDK is installed and configured (`pip install -U gretel-client; gretel configure`).


In [3]:
import json

import yaml
from smart_open import open
import pandas as pd

from gretel_client import submit_docker_local
from gretel_client.projects import create_or_get_unique_project

data_source = "https://gretel-public-website.s3.us-west-2.amazonaws.com/datasets/example-datasets/bike-customer-orders.csv"

# Simple policy to redact PII types with a character.
# Dates are shifted +/- 20 days based on the CustomerID field.
# Income is bucketized to 5000 number increments.

config = """
schema_version: 1.0
models:
  - transforms:
      data_source: "_"
      policies:
        - name: remove_pii
          rules:
            - name: fake_or_redact_pii
              conditions:
                value_label:
                  - person_name
                  - phone_number
                  - gender
                  - birth_date
              transforms:
                - type: redact_with_char
                  attrs:
                    char: X
            - name: dateshifter
              conditions:
                field_label:
                  - date
                  - datetime
                  - birth_date
              transforms:
                - type: dateshift
                  attrs:
                    min: 20
                    max: 20
                    formats: "%Y-%m-%d"
                    field_name: "CustomerID"        
            - name: bucketize-income
              conditions:
                field_name:
                  - YearlyIncome
              transforms:
                - type: numberbucket
                  attrs:
                    min: 0
                    max: 1000000
                    nearest: 5000
"""


In [4]:
# Load and preview the DataFrame to train the transform model on.

df = pd.read_csv(data_source, nrows=500)
df.to_csv("training_data.csv", index=False)
df.head(5)


,CustomerID,Title,FirstName,MiddleName,LastName,Suffix,AddressLine1,AddressLine2,City,StateProvinceName,...,BirthDate,Education,Occupation,Gender,MaritalStatus,HomeOwnerFlag,NumberCarsOwned,NumberChildrenAtHome,TotalChildren,YearlyIncome
0,user_26159,NaN,Virginia,NaN,Raman,NaN,3242 Coralie Drive,NaN,Sydney,New South Wales,...,1957-07-10,Partial College,Skilled Manual,F,S,1,1,0,1,78246
1,user_11748,NaN,Blake,NaN,Hill,NaN,1315 Norse Drive,NaN,Victoria,British Columbia,...,1958-07-18,Graduate Degree,Professional,M,M,1,1,4,5,92987
2,user_25780,NaN,Dana,M,Gill,NaN,Roßstr 9928,NaN,Leipzig,Nordrhein-Westfalen,...,1944-06-05,Partial College,Professional,F,S,0,2,0,5,87805
3,user_16198,NaN,Arthur,NaN,Navarro,NaN,6607 Panoramic Avenue,NaN,Cheltenham,England,...,1972-10-13,High School,Manual,M,S,1,1,0,0,32050
4,user_22931,NaN,Logan,K,Johnson,NaN,4945 Pennsylvania Blvd.,NaN,Caloundra,Queensland,...,1928-03-02,Graduate Degree,Management,M,M,0,1,0,5,136024


In [6]:
project = create_or_get_unique_project(name="local-transform")

In [ ]:
# The following cell will create the transform model and
# run a sample of the data set through the model. this sample
# can be used to ensure the model is functioning correctly
# before continuing.
transform = project.create_model_obj(
    model_config=yaml.safe_load(config), data_source="training_data.csv"
)

run = submit_docker_local(transform, output_dir="tmp/")


In [ ]:
# Review the sampled classification report
# to get an overview of detected data types
report = json.loads(open("tmp/report_json.json.gz").read())
pd.DataFrame(report["metadata"]["fields"])


In [ ]:
# Next let's transform the remaining records using the transformation
# policy and model that was just created.
transform_records = transform.create_record_handler_obj(data_source="training_data.csv")

run = submit_docker_local(
    transform_records, model_path="tmp/model.tar.gz", output_dir="tmp/"
)


In [ ]:
# View the transformation report
report = json.loads(open("tmp/report_json.json.gz").read())
pd.DataFrame(report["metadata"]["fields"])


In [ ]:
# View the transformed data
results = pd.read_csv("tmp/data.gz")
results.head(5)
